In [90]:
# log regression
# feature: start, end, day
# result: duration

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("../data/yellow_tripdata_2020-04.csv", nrows=6000000)

train.info()
train['tpep_pickup_datetime'] = pd.to_datetime(train['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
train['tpep_dropoff_datetime'] = pd.to_datetime(train['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
train['trip_duration'] = (train['tpep_dropoff_datetime'] - train['tpep_pickup_datetime']).dt.seconds
train['PULocationID'].fillna(-1, inplace=True)
train['DOLocationID'].fillna(-1, inplace=True)

train['tpep_pickup_month'] = train['tpep_pickup_datetime'].dt.month
train['tpep_dropoff_month'] = train['tpep_dropoff_datetime'].dt.month
train['tpep_pickup_day_numeric'] = train['tpep_pickup_datetime'].dt.day
train['tpep_dropoff_day_numeric'] = train['tpep_dropoff_datetime'].dt.day
train['tpep_pickup_day_name'] = train['tpep_pickup_datetime'].dt.day_name()
train['tpep_dropoff_day_name'] = train['tpep_dropoff_datetime'].dt.day_name()
train['tpep_pickup_hour'] = train['tpep_pickup_datetime'].dt.hour
train['tpep_dropoff_hour'] = train['tpep_dropoff_datetime'].dt.hour
train['tpep_pickup_day'] = train['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
train['tpep_dropoff_day'] = train['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)

train

/var/folders/wg/62v23f191zsbtpgk23vhgv2h0000gn/T/ipykernel_59455/814977998.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/yellow_tripdata_2020-04.csv", nrows=6000000)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238073 entries, 0 to 238072
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               238073 non-null  int64  
 1   tpep_pickup_datetime   238073 non-null  object 
 2   tpep_dropoff_datetime  238073 non-null  object 
 3   passenger_count        218493 non-null  float64
 4   trip_distance          238073 non-null  float64
 5   RatecodeID             218493 non-null  float64
 6   store_and_fwd_flag     218493 non-null  object 
 7   PULocationID           238073 non-null  int64  
 8   DOLocationID           238073 non-null  int64  
 9   payment_type           238073 non-null  int64  
 10  fare_amount            238073 non-null  float64
 11  extra                  238073 non-null  float64
 12  mta_tax                238073 non-null  float64
 13  tip_amount             238073 non-null  float64
 14  tolls_amount           238073 non-nu

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tpep_pickup_month,tpep_dropoff_month,tpep_pickup_day_numeric,tpep_dropoff_day_numeric,tpep_pickup_day_name,tpep_dropoff_day_name,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_day,tpep_dropoff_day
0,1,2020-04-01 00:41:22,2020-04-01 01:01:53,1.0,1.20,1.0,N,41,24,2,...,4,4,1,1,Wednesday,Wednesday,0,1,3,3
1,1,2020-04-01 00:56:00,2020-04-01 01:09:25,1.0,3.40,1.0,N,95,197,1,...,4,4,1,1,Wednesday,Wednesday,0,1,3,3
2,1,2020-04-01 00:00:26,2020-04-01 00:09:25,1.0,2.80,1.0,N,237,137,1,...,4,4,1,1,Wednesday,Wednesday,0,0,3,3
3,1,2020-04-01 00:24:38,2020-04-01 00:34:38,0.0,2.60,1.0,N,68,142,1,...,4,4,1,1,Wednesday,Wednesday,0,0,3,3
4,2,2020-04-01 00:13:24,2020-04-01 00:18:26,1.0,1.44,1.0,Y,263,74,1,...,4,4,1,1,Wednesday,Wednesday,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238068,2,2020-04-30 23:30:00,2020-04-30 23:39:00,NaN,2.02,NaN,NaN,238,75,0,...,4,4,30,30,Thursday,Thursday,23,23,4,4
238069,2,2020-04-30 23:44:00,2020-04-30 23:54:00,NaN,3.30,NaN,NaN,263,230,0,...,4,4,30,30,Thursday,Thursday,23,23,4,4
238070,2,2020-04-30 23:25:00,2020-04-30 23:38:00,NaN,6.40,NaN,NaN,137,42,0,...,4,4,30,30,Thursday,Thursday,23,23,4,4
238071,2,2020-04-30 23:50:26,2020-05-01 00:28:21,NaN,25.70,NaN,NaN,29,69,0,...,4,5,30,1,Thursday,Friday,23,0,4,5


In [91]:
from sklearn.model_selection import train_test_split

X = train.drop(["fare_amount"], axis=1)
y = train["fare_amount"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [92]:
X_train = X_train[['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour']]
X_test = X_test[['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour']]

In [93]:
from sklearn.metrics import mean_squared_error

avg_fare = round(np.mean(y_train), 2)
baseline_pred = np.repeat(avg_fare, y_test.shape[0])
baseline_rmse = np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :", baseline_rmse)

Basline RMSE of Validation data : 11.443153287956056


In [94]:
# linear regression
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = np.round(lm.predict(X_test), 2)
lm_rmse = np.sqrt(mean_squared_error(y_pred, y_test))
lm_train_rmse = np.sqrt(mean_squared_error(lm.predict(X_train), y_train))
lm_variance = abs(lm_train_rmse - lm_rmse)
print("Test RMSE for Linear Regression is ", lm_rmse)
print("Train RMSE for Linear Regression is ", lm_train_rmse)
print("Variance for Linear Regression is ", lm_variance)

Test RMSE for Linear Regression is  11.396643558887861
Train RMSE for Linear Regression is  11.781743473379455
Variance for Linear Regression is  0.38509991449159386


In [95]:
# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier

rf = RandomForestRegressor(n_estimators=100, random_state=883, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ", rf_rmse)

RMSE for Random Forest is  7.482095927610649


In [96]:
# LightGBM
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
param = {'num_leaves': 31, 'num_trees': 5000, 'objective': 'regression'}
param['metric'] = 'l2_root'
num_round = 5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=10, verbose_eval=20, early_stopping_rounds=20,
                    stratified=False)
lgb_bst = lgb.train(param, train_data, len(cv_results['rmse-mean']))
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse = np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM is ", lgb_rmse)

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:573: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002507 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 5
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 160695, number of used features: 5

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Start training from score 11.654494
[LightGBM] [Info] Start training from score 11.671579
[20]	cv_agg's rmse: 10.0128 + 0.700431
[40]	cv_agg's rmse: 9.63526 + 0.731158
[60]	cv_agg's rmse: 9.4306 + 0.737494
[80]	cv_agg's rmse: 9.28397 + 0.74763
[100]	cv_agg's rmse: 9.17419 + 0.754623
[120]	cv_agg's rmse: 9.10296 + 0.761966
[140]	cv_agg's rmse: 9.0418 + 0.762081
[160]	cv_agg's rmse: 8.98755 + 0.766219
[180]	cv_agg's rmse: 8.93554 + 0.769806
[200]	cv_agg's rmse: 8.89103 + 0.77386
[220]	cv_agg's rmse: 8.84857 + 0.773438
[240]	cv_agg's rmse: 8.81117 + 0.775543
[260]	cv_agg's rmse: 8.77849 + 0.782038
[280]	cv_agg's rmse: 8.74711 + 0.78437
[300]	cv_agg's rmse: 8.71871 + 0.784278
[320]	cv_agg's rmse: 8.69812 + 0.78405
[340]	cv_agg's rmse: 8.67522 + 0.782848
[360]	cv_agg's rmse: 8.65348 + 0.780715
[380]	cv_agg's rmse: 8.6337 + 0.783345
[400]	cv_agg's rmse: 8.61796 + 0.786183
[420]	cv_agg's rmse: 8.602 + 0.786867
[440]	cv_agg's rmse: 8.58746 + 0.788638
[460]	cv_agg's rmse: 8.57

/Users/kenoc1/.conda/envs/taxi2/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 178554, number of used features: 5
[LightGBM] [Info] Start training from score 11.660754
RMSE for Light GBM is  7.5119101475383765


In [99]:
# testing 1 dataset
df_test = pd.DataFrame([[107, 87, 4, 6, 14]], columns=['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour'])

y_test_pred = np.round(lm.predict(df_test), 2)
rf_test_pred = rf.predict(df_test)
lgb_test_pred = lgb_bst.predict(df_test)
#print(df_test)
print("Linear Regression: ", y_test_pred)
print("Random Forest: ", rf_test_pred)
print("LightGBM: ", lgb_test_pred)

Linear Regression:  [12.49]
Random Forest:  [13.12443333]
LightGBM:  [16.86687947]
